In [ ]:
import os
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
YOUR_WAVE = 'audio.wav'

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(YOUR_WAVE, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
from google.cloud import speech, storage

YOUR_SERVICE = 'YOUR_SERVICE'
YOUR_BUCKET = 'YOUR_BUCKET'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
storage_client = storage.Client()
speech_client = speech.SpeechClient()

# upload file to GCS(Google Cloud Storage)
bucket = storage_client.bucket(YOUR_BUCKET)
bucket.blob(YOUR_WAVE).upload_from_filename(YOUR_WAVE)
uri = f'gs://{YOUR_BUCKET}/{YOUR_WAVE}'

# Transcript the audio
audio = speech.RecognitionAudio(uri=uri)
config = speech.RecognitionConfig(
    sample_rate_hertz=RATE,
    audio_channel_count=CHANNELS,  # take care, default is 1
    language_code="zh-TW"
)
response = speech_client.recognize(config=config, audio=audio)

for result in response.results:
    print(f'{result.alternatives[0].transcript}, {result.alternatives[0].confidence:.3f}')